In [ ]:
try:
    import elasticsearch
    from elasticsearch import Elasticsearch
    
    import pandas as pd
    import json
    from ast import literal_eval
    from tqdm import tqdm
    import datetime
    import os
    import sys
    
    import os
    import tensorflow.compat.v1 as tf
    tf.disable_v2_behavior()
    import tensorflow_hub as hub
    import numpy as np
    
    from elasticsearch import helpers
    print("Loaded  .. . . . . . . .")
except Exception as E:
    print("Some Modules are Missing {} ".format(E))

In [ ]:
tf.__version__


In [ ]:
df = pd.read_csv("val_knowledgebase.csv")
print("read")

In [ ]:
df.head()

In [ ]:
graph = tf.Graph()
with tf.Session(graph = graph) as session:
    print("Downloading pre-trained embeddings from tensorflow hub…")
    embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder/2")
    text_ph = tf.placeholder(tf.string)
    embeddings = embed(text_ph)
    print("Done.")
    print("Creating tensorflow session…")
    session = tf.Session()
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    print("Done.")

In [ ]:
def embed_text(text):
    text=[text]
    vectors = session.run(embeddings, feed_dict={text_ph: text})
    temp= [vector.tolist() for vector in vectors]
    return temp[0]

In [ ]:
df["universal_vector_question"] = df["query"].apply(embed_text)

In [ ]:
df["universal_vector_answer"] = df["atom"].apply(embed_text)

In [ ]:
df.head()

In [ ]:
qu=df["universal_vector_question"]

In [ ]:
an=df["universal_vector_answer"]

In [ ]:
len(an)

In [ ]:
df["universal_vector_question"].to_pickle("/home/srinivas/Desktop/validation_arrays/use/universal_question_series.pkl")

In [ ]:
df["universal_vector_answer"].to_pickle("/home/srinivas/Desktop/validation_arrays/use/universal_answer_series.pkl")

In [ ]:
import pickle
with open('/home/srinivas/Desktop/validation_arrays/use/universal_question_series.pkl','rb') as f:
     use_ans77 = pickle.load(f)

In [ ]:
len(use_ans77)

In [ ]:
len(df["universal_vector"][0])

In [ ]:
Settings = {
   "settings":{
      "number_of_shards":1,
      "number_of_replicas":0
   },
   "mappings":{
       "dynamic": "true",
        "_source": {
        "enabled": "true"
        },
       "properties":{
            "answer": {
            "type": "text"
            },
          "universal_vector":{
         "type":"dense_vector",
         "dims":512
      }, 
            "country":{  
            "type":"keyword"
         },   
          "product":{  
            "type":"keyword"
         }
      
    }
   }
}

In [ ]:
ENDPOINT = "http://localhost:9200/"
es = Elasticsearch(timeout=600,hosts=ENDPOINT)
print("python client created")

In [ ]:
#creating the database
IndexName = 'use_vector1'
my = es.indices.create(index=IndexName, ignore=[400,404], body=Settings)

In [ ]:
my

In [ ]:
df.columns

In [ ]:
df22 = df.to_dict('records')
df22

In [ ]:
arr=[]
for c, line in enumerate(df22):
    arr.append({"answer":line.get("para_text",""),
    'universal_vector':line.get('universal_vector', ""),
    'country':line.get("country","").lower().strip(),
    'product':line.get("Product name","").lower().strip()})


In [ ]:
arr

In [ ]:
#inserting into the database
#doc_type similar to table in rdbms

for i in range(0,len(arr)):

    resp=es.index(index=IndexName,doc_type="_doc",id=i,body=arr[i])

In [ ]:
resp

In [ ]:
es.indices.get_mapping(index=IndexName)

In [ ]:
#testing
que = "what is ATACAND used for in america?"

In [ ]:
que_vec=embed_text(que)

In [ ]:
que_vec

In [ ]:
len(que_vec)

In [ ]:
temp_prod=df["Product name"]
temp_country=df["country"]

In [ ]:
set(temp_prod)

In [ ]:
set(temp_country)

In [ ]:

script_query = {
"script_score": {
"query": {"term":{  
         "country":{  
            "value":"canada"
         }},
      "term":{"product":{
             "value":"candesartan cilexetil"   
            }}},
"script": {
"source": "Math.abs(cosineSimilarity(params.query_vector, doc['universal_vector']))",
"params": {"query_vector": que_vec }
}}}

In [ ]:
ans=es.search(index=IndexName,body={"size": 10,"query": script_query,"_source": {"includes": ["answer"]}})
ans

In [ ]:
def questions():
    count=0
    que_arr=[]
    for x in df["Possible Questions"]:
        sent=x.split("\n")
        for q in sent:
            if re.search('[a-zA-Z]',q)!=None:
                que_arr.append([q,count])
        count+=1
    return que_arr

In [ ]:
import re

In [ ]:
que_arr=questions()

In [ ]:
que_arr

In [ ]:
len(que_arr)

In [ ]:
def rank(ans):
    required=ans['hits']
    main_arr=required["hits"]
    count=1
    ids__=dict()
    for x in main_arr:
        id_no= x["_id"]

        score=x["_score"]

        answer_=x["_source"]["answer"]

        ids__[id_no.strip()]=1/count
        count+=1
    return ids__
    

In [ ]:
ranker=rank(ans)

In [ ]:
ranker

In [ ]:
total=0
count=0
recall_count=0
for x in que_arr:
    question=x[0].strip()
    print(question)
    id_number=str(x[1])
    que_vec=embed_text(question)
    #print(que_vec)
    script_query = {
    "script_score": {
    "query": {"term":{  
             "country":{  
                "value":"canada"
             }},
          "term":{"product":{
                 "value":"candesartan cilexetil"   
                }}},
    "script": {
    "source": "Math.abs(cosineSimilarity(params.query_vector, doc['universal_vector']))",
    "params": {"query_vector": que_vec}
    }}}
    ans=es.search(index=IndexName,body={"size": 10,"query": script_query,"_source": {"includes": ["answer"]}})
    ranker=rank(ans)
    if id_number in ranker:
        total+=ranker[id_number]
        print(ranker[id_number])
        recall_count+=1
    else:
        print(0)
    count+=1

print("mrr is"+str(total/count))
print("recall is"+str(recall_count/count))